In [1]:
from random import randint

In [2]:
key = (randint(0,15), randint(0,15))
print(key)

(9, 1)


In [3]:
sBox = [12, 5, 6, 11, 9, 0, 10, 13, 3, 14, 15, 8, 4, 7, 1, 2]
print(sBox)

[12, 5, 6, 11, 9, 0, 10, 13, 3, 14, 15, 8, 4, 7, 1, 2]


In [4]:
msg = 10

In [5]:
def round(msg, key):
    return sBox[msg^key]

In [6]:
def enc(msg, key):
    return round(round(msg, key[0]), key[1])

In [7]:
xobs = [sBox.index(i) for i in range(0, 16)]
print(xobs)

[5, 14, 15, 8, 12, 1, 2, 13, 11, 4, 6, 3, 0, 7, 9, 10]


In [8]:
def back_round(msg, key):
    return xobs[msg]^key

In [9]:
def dec(msg, key):
    return back_round(back_round(msg, key[1]), key[0])

In [10]:
print(msg == dec(enc(msg, key), key))

True


In [11]:
bin(255)

'0b11111111'

In [12]:
s = "hello"
for x in bytearray(s, 'utf-8'):
    print(x)

104
101
108
108
111


In [13]:
bin(104>>4<<4|104&0b1111)

'0b1101000'

In [14]:
ord('a')


97

In [15]:
chr(122)

'z'

In [16]:
with open("message.txt", mode='rb') as file: # b is important -> binary
    fileContent = file.read()
    print(fileContent+fileContent)
    for f in fileContent[:-1]:
        print(chr(f))

b'coco\ncoco\n'
c
o
c
o


In [17]:
a = b""
print(a)

b''


In [18]:
bytes([122])

b'z'

In [19]:
with open("message.enc", mode='rb') as file: # b is important -> binary
    fileContent = file.read()
    for f in fileContent:
        print(chr(f))

¯
»

3
{
Õ


In [20]:
from random import randint


class ToyCipher():
	def __init__(self, sBox):
		self.sBox = sBox
		self.xobs = [sBox.index(i) for i in range(0, 16)]
	
	def round(self, msg, key):
		return self.sBox[msg^key]
    	
	def back_round(self, msg, key):
		return self.xobs[msg]^key
    	
	def enc(self, msg, key):
		return self.round(self.round(msg, key[0]), key[1])
    
	def dec(self, msg, key):
		return self.back_round(self.back_round(msg, key[1]), key[0])
		
	def enc_byte(self, msg, key):
		return self.enc(msg>>4, key)<<4|self.enc(msg&0b1111, key)
		
	def dec_byte(self, msg, key):
		return self.dec(msg>>4, key)<<4|self.dec(msg&0b1111, key)
		
	def enc_char(self, msg, key):
		return chr(self.enc_byte(ord(msg), key))
		
	def dec_char(self, msg, key):
		return chr(self.dec_byte(ord(msg), key))
		
	def enc_string(self, msg, key):
		encrypted = ""
		for char in msg:
			encrypted += self.enc_char(char, key)
		return encrypted
		
	def dec_string(self, msg, key):
		decrypted = ""
		for char in msg:
			decrypted += self.dec_char(char, key)
		return decrypted
		
	def enc_file(self, fileName, encFileName, key):
		encrypted = b""
		with open(fileName, mode='rb') as file: # b is important -> binary
			fileContent = file.read()
		for f in fileContent:
			encrypted += bytes([self.enc_byte(f, key)])
		with open(encFileName, mode='wb') as file:
			file.write(encrypted)
			
	def dec_file(self, encFileName, decFileName, key):
		decrypted = ""
		with open(encFileName, mode='rb') as file: # b is important -> binary
			fileContent = file.read()
		for f in fileContent:
			decrypted += chr(self.dec_byte(f, key))
		with open(decFileName, mode='w') as file:
			file.write(decrypted)
			
	def enc_cbc_file(self, fileName, encFileName, key):
		iv = randint(0, 255)
		encrypted = bytes([iv])
		with open(fileName, mode='rb') as file: # b is important -> binary
			fileContent = file.read()
		for f in fileContent:
			iv = self.enc_byte(f^iv, key)
			encrypted += bytes([iv])
		with open(encFileName, mode='wb') as file:
			file.write(encrypted)
			
	def dec_cbc_file(self, encFileName, decFileName, key):
		decrypted = ""
		with open(encFileName, mode='rb') as file: # b is important -> binary
			iv = file.read(1)
			iv = iv[0]
			file.seek(1)
			fileContent = file.read()
		for f in fileContent:
			decrypted += chr(self.dec_byte(f, key)^iv)
			iv = f
		with open(decFileName, mode='w') as file:
			file.write(decrypted)
			
	def enc_cfb_file(self, fileName, encFileName, key):
		iv = randint(0, 255)
		encrypted = bytes([iv])
		with open(fileName, mode='rb') as file: # b is important -> binary
			fileContent = file.read()
		for f in fileContent:
			iv = self.enc_byte(iv, key)^f
			encrypted += bytes([iv])
		with open(encFileName, mode='wb') as file:
			file.write(encrypted)
			
	def dec_cfb_file(self, encFileName, decFileName, key):
		decrypted = ""
		with open(encFileName, mode='rb') as file: # b is important -> binary
			iv = file.read(1)
			iv = iv[0]
			file.seek(1)
			fileContent = file.read()
		for f in fileContent:
			decrypted += chr(self.enc_byte(iv, key)^f)
			iv = f
		with open(decFileName, mode='w') as file:
			file.write(decrypted)
			
	def enc_ofb_file(self, fileName, encFileName, key):
		iv = randint(0, 255)
		encrypted = bytes([iv])
		with open(fileName, mode='rb') as file: # b is important -> binary
			fileContent = file.read()
		for f in fileContent:
			encrypted += bytes([self.enc_byte(iv, key)^f])
		with open(encFileName, mode='wb') as file:
			file.write(encrypted)
			
	def dec_ofb_file(self, encFileName, decFileName, key):
		decrypted = ""
		with open(encFileName, mode='rb') as file: # b is important -> binary
			iv = file.read(1)
			iv = iv[0]
			file.seek(1)
			fileContent = file.read()
		for f in fileContent:
			decrypted += chr(self.enc_byte(iv, key)^f)
		with open(decFileName, mode='w') as file:
			file.write(decrypted)
			

# attact par anayse de fréquance

key = (9, 0)
print('key : k0 = {0}, k1 = {1}'.format(key[0], key[1]))

sBox = [12, 5, 6, 11, 9, 0, 10, 13, 3, 14, 15, 8, 4, 7, 1, 2]
print('sBox = {0}'.format(sBox))


key : k0 = 9, k1 = 0
sBox = [12, 5, 6, 11, 9, 0, 10, 13, 3, 14, 15, 8, 4, 7, 1, 2]


In [30]:
TC = ToyCipher(sBox)
TC.enc_cbc_file("message.txt", "message.enc", key)
TC.dec_cbc_file("message.enc", "message.dec", key)

In [31]:
with open("message.enc", mode='rb') as file: # b is important -> binary
    fileContent = file.read()
    decmessage = ""
    for f in fileContent:
        decmessage += chr(f)
    print(decmessage)

¾î`¹


In [32]:
with open("message.dec", mode='rb') as file: # b is important -> binary
    fileContent = file.read()
    decmessage = ""
    for f in fileContent:
        decmessage += chr(f)
    print(decmessage)

coco

